<a href="https://colab.research.google.com/github/mariacmartins/bioinformatica-disciplina/blob/main/Preditor_de_localiza%C3%A7%C3%A3o_de_proteinas_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 9.1MB/s 


In [2]:
#Obtendo os dados
!wget -O membrane.fasta 'https://www.uniprot.org/uniprot/?query=taxonomy%3Abacteria+locations%3A%28location%3Amembrane%29&sort=score&format=fasta&limit=10000'
!wget -O cytoplasm.fasta 'https://www.uniprot.org/uniprot/?query=taxonomy%3Abacteria+locations%3A%28location%3Acytoplasm%29&sort=score&format=fasta&limit=10000'

--2021-04-12 16:11:47--  https://www.uniprot.org/uniprot/?query=taxonomy%3Abacteria+locations%3A%28location%3Amembrane%29&sort=score&format=fasta&limit=10000
Resolving www.uniprot.org (www.uniprot.org)... 193.62.193.81, 128.175.245.202
Connecting to www.uniprot.org (www.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘membrane.fasta’

membrane.fasta          [      <=>           ]   5.68M   306KB/s    in 17s     

2021-04-12 16:12:18 (345 KB/s) - ‘membrane.fasta’ saved [5952076]

--2021-04-12 16:12:18--  https://www.uniprot.org/uniprot/?query=taxonomy%3Abacteria+locations%3A%28location%3Acytoplasm%29&sort=score&format=fasta&limit=10000
Resolving www.uniprot.org (www.uniprot.org)... 128.175.245.202, 193.62.193.81
Connecting to www.uniprot.org (www.uniprot.org)|128.175.245.202|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘cytoplasm.fasta’



In [3]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam
import pandas as pd

In [4]:
def compute_aa_composition(protein_sequence):
  protein_analyzer = ProtParam.ProteinAnalysis(str(protein_sequence))
  protein_data = protein_analyzer.get_amino_acids_percent()
  return protein_data

In [5]:
#Construindo os dataframes com localização de proteínas - membrana e citoplasma
df = pd.DataFrame()

handle = open('membrane.fasta')
parser = SeqIO.parse(handle, 'fasta')

for record in parser:
  protein_data = compute_aa_composition(record.seq)
  protein_data['membrane'] = 1
  df = df.append(protein_data, ignore_index=True)


handle = open('cytoplasm.fasta')
parser = SeqIO.parse(handle, 'fasta')

for record in parser:
  protein_data = compute_aa_composition(record.seq)
  protein_data['membrane'] = 0
  df = df.append(protein_data, ignore_index=True)

df.head(10)

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,membrane
0,0.308789,0.004751,0.047506,0.106888,0.014252,0.049881,0.002375,0.033254,0.156770,0.045131,0.011876,0.026128,0.023753,0.047506,0.021378,0.052257,0.014252,0.019002,0.002375,0.011876,1.0
1,0.072368,0.006579,0.085526,0.052632,0.026316,0.085526,0.026316,0.085526,0.046053,0.072368,0.032895,0.006579,0.059211,0.032895,0.065789,0.046053,0.039474,0.105263,0.026316,0.026316,1.0
2,0.039340,0.004230,0.075296,0.070643,0.055415,0.054569,0.009729,0.104907,0.076988,0.085871,0.014805,0.096870,0.024112,0.024958,0.019459,0.076988,0.051607,0.047377,0.006768,0.060068,1.0
3,0.143204,0.021845,0.019417,0.016990,0.067961,0.131068,0.004854,0.060680,0.036408,0.109223,0.041262,0.009709,0.063107,0.019417,0.026699,0.036408,0.063107,0.050971,0.033981,0.043689,1.0
4,0.111597,0.007659,0.050328,0.063457,0.020788,0.050328,0.016411,0.060175,0.025164,0.145514,0.021882,0.038293,0.033917,0.094092,0.066740,0.066740,0.055799,0.049234,0.008753,0.013129,1.0
5,0.110687,0.011450,0.053435,0.091603,0.019084,0.072519,0.026718,0.049618,0.015267,0.110687,0.015267,0.022901,0.041985,0.019084,0.099237,0.064885,0.038168,0.072519,0.022901,0.041985,1.0
6,0.061571,0.002123,0.076433,0.063694,0.029724,0.055202,0.016985,0.048832,0.065817,0.087049,0.012739,0.053079,0.036093,0.044586,0.042463,0.074310,0.076433,0.095541,0.016985,0.040340,1.0
7,0.101031,0.014433,0.035052,0.020619,0.092784,0.084536,0.020619,0.068041,0.028866,0.140206,0.024742,0.016495,0.041237,0.018557,0.035052,0.061856,0.068041,0.080412,0.020619,0.026804,1.0
8,0.048110,0.010309,0.037801,0.027491,0.127148,0.072165,0.017182,0.061856,0.044674,0.158076,0.013746,0.030928,0.037801,0.006873,0.037801,0.089347,0.037801,0.099656,0.017182,0.024055,1.0
9,0.084722,0.004167,0.051389,0.062500,0.026389,0.062500,0.009722,0.069444,0.061111,0.105556,0.023611,0.050000,0.036111,0.051389,0.050000,0.068056,0.066667,0.081944,0.006944,0.027778,1.0


In [6]:
X = df.drop(['membrane'], axis=1)
y = df['membrane']

In [7]:
X

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,0.308789,0.004751,0.047506,0.106888,0.014252,0.049881,0.002375,0.033254,0.156770,0.045131,0.011876,0.026128,0.023753,0.047506,0.021378,0.052257,0.014252,0.019002,0.002375,0.011876
1,0.072368,0.006579,0.085526,0.052632,0.026316,0.085526,0.026316,0.085526,0.046053,0.072368,0.032895,0.006579,0.059211,0.032895,0.065789,0.046053,0.039474,0.105263,0.026316,0.026316
2,0.039340,0.004230,0.075296,0.070643,0.055415,0.054569,0.009729,0.104907,0.076988,0.085871,0.014805,0.096870,0.024112,0.024958,0.019459,0.076988,0.051607,0.047377,0.006768,0.060068
3,0.143204,0.021845,0.019417,0.016990,0.067961,0.131068,0.004854,0.060680,0.036408,0.109223,0.041262,0.009709,0.063107,0.019417,0.026699,0.036408,0.063107,0.050971,0.033981,0.043689
4,0.111597,0.007659,0.050328,0.063457,0.020788,0.050328,0.016411,0.060175,0.025164,0.145514,0.021882,0.038293,0.033917,0.094092,0.066740,0.066740,0.055799,0.049234,0.008753,0.013129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.144737,0.010965,0.050439,0.074561,0.024123,0.105263,0.019737,0.067982,0.041667,0.078947,0.015351,0.030702,0.032895,0.015351,0.070175,0.052632,0.046053,0.094298,0.004386,0.019737
19996,0.078431,0.015251,0.047930,0.076253,0.021786,0.089325,0.034858,0.080610,0.071895,0.067538,0.015251,0.056645,0.026144,0.037037,0.028322,0.065359,0.069717,0.089325,0.000000,0.028322
19997,0.085153,0.017467,0.045852,0.080786,0.019651,0.093886,0.026201,0.069869,0.069869,0.063319,0.010917,0.048035,0.032751,0.048035,0.037118,0.050218,0.069869,0.104803,0.000000,0.026201
19998,0.123620,0.011038,0.072848,0.041943,0.015453,0.103753,0.019868,0.057395,0.039735,0.088300,0.006623,0.046358,0.039735,0.037528,0.057395,0.028698,0.075055,0.108168,0.006623,0.019868


In [8]:
y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
19995    0.0
19996    0.0
19997    0.0
19998    0.0
19999    0.0
Name: membrane, Length: 20000, dtype: float64

### Comparando modelos de predição de localização subcelular de proteínas por Machine Learning

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.88      0.79      0.83      2540
         1.0       0.80      0.89      0.84      2460

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000



In [12]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [13]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.93      0.92      0.93      2540
         1.0       0.92      0.93      0.93      2460

    accuracy                           0.93      5000
   macro avg       0.93      0.93      0.93      5000
weighted avg       0.93      0.93      0.93      5000



In [14]:
model = MLPClassifier(hidden_layer_sizes=(100, 50, 25))
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 50, 25), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [15]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      2540
         1.0       0.95      0.95      0.95      2460

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



In [16]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [17]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.92      0.91      0.91      2540
         1.0       0.90      0.91      0.91      2460

    accuracy                           0.91      5000
   macro avg       0.91      0.91      0.91      5000
weighted avg       0.91      0.91      0.91      5000



In [18]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94      2540
         1.0       0.93      0.95      0.94      2460

    accuracy                           0.94      5000
   macro avg       0.94      0.94      0.94      5000
weighted avg       0.94      0.94      0.94      5000



In [20]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [21]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93      2540
         1.0       0.92      0.94      0.93      2460

    accuracy                           0.93      5000
   macro avg       0.93      0.93      0.93      5000
weighted avg       0.93      0.93      0.93      5000

